<h1 align="center"><b>AI Lab: Computer Vision and NLP</b></h1>
<h3 align="center">Lessons 17: Artificial Neural Networks</h3>

---

Our brain works with neurons, which are small cells which can retain data and make reasonments. Computer scientists have tried along the past years to replicate a human brain and thus allow a computer to have an artificial one. Since neurons work with small quantities of electricity, then such small pulses of energy could be translated with a binary pulse. One of the first applications of a neuron was to implement logic gates, such as the `AND` ($y = x_1 \wedge x_2$) or the `OR` ($y = x_1 \vee x_2$).

An artificial neuron is very similar to a logic gate: with an `AND` gate with 2 inputs $x_1$ and $x_2$, we would have that the output $y$ is equal to 1 if, by summing the two inputs, we reach a threshold $T \geq 2$. The neuron has, instead of a fixed threshold, a variable one. Such threshold represents thus the **activation function** of a neuron.

We can compare how powerful tools such as `pytorch` are compared to other tools such as `scikit-learn`:

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

We now proceed to load the dataset, and thanks to the passing of the option `return_X_y = true` we can tell `scikit-learn` to pass us the dataset already splitted:

In [7]:
#x, y = fetch_openml('mnist_784', return_X_y=True)
digits = load_digits()
x, y = digits.data, digits.target

Now, we can create the test and train datasets:

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

Now we can create a multilevel perceptron (**MLP**), and with `scikit-learn` we can provide some hidden layers which are not considering the input and output layers:

In [9]:
model = MLPClassifier(hidden_layer_sizes=(20,))

We can now train the model:

In [10]:
model.fit(x_train, y_train)

/Users/leonardo/miniconda3/envs/ailab/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20,))

And finally, we can test the model and see how well it performs:

In [12]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.9629629629629629


---

Now, this was the version with `scikit-learn`. We said thought previously that `scikit-learn` runs only on CPUs, and this is not helpful while trying to train a large model. So we can use instead `pytorch`, and we can try to do it all over again from scratch. The problem with `pytorch` is that we have to do everything from scratch. In order to use `pytorch`, we start by importing the package:

In [16]:
import torch
# from torch.utils import Dataloader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
import torchmetrics

Let's import a dataset with some images:

In [18]:
training_data = datasets.FashionMNIST(
    root="pytorch_datasets",
    train=True,
    download=True,
    transform=ToTensor() # This function will transform the data into tensors,
                         # we just need to specify the transformation function
) # type: ignore

test_data = datasets.FashionMNIST(
    root="pytorch_datasets",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:57<00:00, 461238.47it/s] 


Extracting pytorch_datasets/FashionMNIST/raw/train-images-idx3-ubyte.gz to pytorch_datasets/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 55923.85it/s]


Extracting pytorch_datasets/FashionMNIST/raw/train-labels-idx1-ubyte.gz to pytorch_datasets/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:10<00:00, 441295.09it/s]


Extracting pytorch_datasets/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to pytorch_datasets/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5958133.83it/s]


Extracting pytorch_datasets/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to pytorch_datasets/FashionMNIST/raw



Now, here it comes the most important part: we have to fetch the GPU in order to use it instead of the CPU:

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

We can now start to create our multilevel perceptron:

In [20]:
class OurMLP(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        pass